# 🦾 MILESTONE 2: AVENGERS FACE RECOGNITION SYSTEM
## Trusted User Detection & Enrollment

**Objective**: Detect and recognize trusted users (you, roommates, friends)

**Features**:
- Face detection using MediaPipe/face_recognition
- Enrollment system for trusted faces
- Real-time recognition from webcam
- Avengers-themed welcome messages

In [1]:

# Cell 2: Install Dependencies
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("📦 Installing face recognition packages...")
    # !apt-get -qq install cmake
    # !pip install -q face-recognition opencv-python mediapipe pillow
else:
    print("💻 Running locally. Ensure requirements.txt is installed.")

# Cell 3: Import Libraries
import cv2
import face_recognition
import numpy as np
import pickle
import os
from pathlib import Path
from datetime import datetime
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass
import json
import matplotlib.pyplot as plt
from PIL import Image
import time

print("✅ All imports successful!")


💻 Running locally. Ensure requirements.txt is installed.
✅ All imports successful!


In [2]:
# Cell 4: Data Classes and Configuration
@dataclass
class TrustedPerson:
    """Represents a trusted individual"""
    name: str
    role: str  # e.g., "owner", "roommate", "friend"
    face_encoding: np.ndarray
    enrolled_date: str
    photo_path: str
    recognition_count: int = 0
    
    def to_dict(self):
        """Convert to dictionary for JSON serialization"""
        return {
            'name': self.name,
            'role': self.role,
            'face_encoding': self.face_encoding.tolist(),
            'enrolled_date': self.enrolled_date,
            'photo_path': self.photo_path,
            'recognition_count': self.recognition_count
        }
    
    @staticmethod
    def from_dict(data):
        """Create from dictionary"""
        return TrustedPerson(
            name=data['name'],
            role=data['role'],
            face_encoding=np.array(data['face_encoding']),
            enrolled_date=data['enrolled_date'],
            photo_path=data['photo_path'],
            recognition_count=data.get('recognition_count', 0)
        )

class FaceRecognitionConfig:
    """Configuration for face recognition system"""
    
    # Paths
    DATA_DIR = Path("data")
    FACES_DIR = DATA_DIR / "trusted_faces" / "photos"
    EMBEDDINGS_DIR = DATA_DIR / "trusted_faces" / "embeddings"
    DB_FILE = EMBEDDINGS_DIR / "trusted_persons.pkl"
    
    # Recognition parameters
    RECOGNITION_TOLERANCE = 0.6  # Lower = stricter (0.4-0.7 recommended)
    FACE_DETECTION_MODEL = "hog"  # "hog" (faster, CPU) or "cnn" (accurate, GPU)
    MIN_FACE_SIZE = 50  # Minimum face size in pixels
    
    # Avengers personality mappings
    PERSONALITY_GREETINGS = {
        "owner": [
            "Welcome back, boss. JARVIS has kept everything secure.",
            "Good to see you, sir. All systems nominal.",
            "Access granted. Room status: secured."
        ],
        "roommate": [
            "Hey roommate! Everything's been quiet here.",
            "Welcome back. No intruders detected during your absence.",
            "Access granted. Room secured as usual."
        ],
        "friend": [
            "Hello, friend! You're cleared to enter.",
            "Welcome! JARVIS recognizes you from my database.",
            "Access granted. Good to see a familiar face."
        ]
    }
    
    INTRUDER_MESSAGES = [
        "Unrecognized individual detected. Please identify yourself.",
        "Hold on there, stranger. Who are you and what's your business here?",
        "Access denied. You are not in my database of trusted individuals."
    ]
    
    @classmethod
    def setup_directories(cls):
        """Create necessary directories"""
        cls.FACES_DIR.mkdir(parents=True, exist_ok=True)
        cls.EMBEDDINGS_DIR.mkdir(parents=True, exist_ok=True)
        print(f"✅ Directories created at {cls.DATA_DIR}")

config = FaceRecognitionConfig()
config.setup_directories()


✅ Directories created at data


In [3]:
# Cell 5: Face Enrollment System (CORRECTED)
class FaceEnrollmentSystem:
    """Handles enrollment of trusted faces"""
    
    def __init__(self):
        self.config = FaceRecognitionConfig()
        self.trusted_persons: Dict[str, TrustedPerson] = {}
        self.load_database()
    
    def enroll_from_image(self, image_path: str, name: str, role: str = "friend") -> bool:
        """
        Enroll a person from an image file
        
        Args:
            image_path: Path to the image file
            name: Person's name
            role: Role (owner/roommate/friend)
        
        Returns:
            True if enrollment successful
        """
        print(f"\n{'='*60}")
        print(f"📸 ENROLLING: {name} ({role})")
        print(f"{'='*60}")
        
        try:
            # Load image
            image = face_recognition.load_image_file(image_path)
            print(f"✅ Image loaded: {image.shape}")
            
            # Detect faces
            face_locations = face_recognition.face_locations(
                image, 
                model=self.config.FACE_DETECTION_MODEL
            )
            
            if len(face_locations) == 0:
                print("❌ No faces detected in image!")
                return False
            
            if len(face_locations) > 1:
                print(f"⚠️  Multiple faces detected ({len(face_locations)}). Using the largest face.")
                # Use the largest face
                face_locations = [max(face_locations, key=lambda loc: (loc[2] - loc[0]) * (loc[1] - loc[3]))]
            
            # Get face encoding
            face_encodings = face_recognition.face_encodings(image, face_locations)
            
            if len(face_encodings) == 0:
                print("❌ Could not generate face encoding!")
                return False
            
            face_encoding = face_encodings[0]
            
            # Create trusted person object
            person = TrustedPerson(
                name=name,
                role=role,
                face_encoding=face_encoding,
                enrolled_date=datetime.now().isoformat(),
                photo_path=image_path
            )
            
            # Save to database
            self.trusted_persons[name] = person
            self.save_database()
            
            print(f"✅ {name} enrolled successfully!")
            print(f"   Role: {role}")
            print(f"   Encoding shape: {face_encoding.shape}")
            print(f"{'='*60}\n")
            
            return True
            
        except Exception as e:
            print(f"❌ Enrollment failed: {e}")
            return False
    
    def enroll_from_webcam(self, name: str, role: str = "friend", num_samples: int = 3) -> bool:
        """
        AUTO-CAPTURE VERSION - No space bar needed!
        Just look at camera and stay still for 6 seconds
        
        Args:
            name: Person's name
            role: Role (owner/roommate/friend)
            num_samples: Number of samples to capture (default 3)
        
        Returns:
            True if enrollment successful
        """
        print(f"\n{'='*60}")
        print(f"📹 AUTO-ENROLLMENT: {name} as {role}")
        print(f"{'='*60}")
        print("📋 Just look at the camera and STAY STILL")
        print("📋 System captures automatically every 2 seconds")
        print("📋 Press ESC to cancel\n")
        
        # Open webcam
        video_capture = cv2.VideoCapture(0)
        time.sleep(1)  # Let camera warm up
        
        if not video_capture.isOpened():
            print("❌ Cannot open webcam!")
            return False
        
        print("✅ Webcam opened")
        
        captured_encodings = []
        sample_count = 0
        last_capture = 0
        
        try:
            while sample_count < num_samples:
                ret, frame = video_capture.read()
                
                if not ret:
                    print("⚠️ Can't read frame, retrying...")
                    time.sleep(0.1)
                    continue
                
                # Show frame
                display_frame = frame.copy()
                
                # Add text overlay
                cv2.putText(display_frame, f"Captured: {sample_count}/{num_samples}", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(display_frame, "Look at camera & stay still", 
                           (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                cv2.imshow('Enrollment - Auto Capture', display_frame)
                
                # Process keyboard
                key = cv2.waitKey(30) & 0xFF
                if key == 27:  # ESC
                    print("\n❌ Cancelled by user")
                    video_capture.release()
                    cv2.destroyAllWindows()
                    return False
                
                # Try to capture every 2 seconds
                current_time = time.time()
                if current_time - last_capture >= 2.0:
                    
                    print(f"📸 Attempting capture {sample_count + 1}...")
                    
                    # Convert to RGB
                    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    
                    # Detect faces (using fast HOG model)
                    print("   Detecting faces...")
                    face_locations = face_recognition.face_locations(rgb_frame, model="hog")
                    
                    print(f"   Found {len(face_locations)} face(s)")
                    
                    if len(face_locations) == 1:
                        # Perfect - one face!
                        print("   Getting face encoding...")
                        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
                        
                        if len(face_encodings) > 0:
                            # Success!
                            captured_encodings.append(face_encodings[0])
                            sample_count += 1
                            last_capture = current_time
                            
                            # Save photo
                            photo_path = self.config.FACES_DIR / f"{name}_{sample_count}.jpg"
                            cv2.imwrite(str(photo_path), frame)
                            
                            print(f"   ✅ Sample {sample_count}/{num_samples} captured!\n")
                        else:
                            print("   ⚠️ Could not encode face, try again...")
                    
                    elif len(face_locations) == 0:
                        print("   ⚠️ No face detected - move closer or improve lighting\n")
                    else:
                        print(f"   ⚠️ {len(face_locations)} faces detected - only one person please\n")
                    
                    last_capture = current_time  # Update to avoid spam
            
            # Close webcam
            print("\n📷 Closing webcam...")
            video_capture.release()
            cv2.destroyAllWindows()
            time.sleep(0.5)
            print("✅ Webcam closed\n")
            
            # Save to database
            if len(captured_encodings) >= num_samples:
                print("💾 Saving to database...")
                
                # Average the encodings
                avg_encoding = np.mean(captured_encodings, axis=0)
                
                # Create person object
                person = TrustedPerson(
                    name=name,
                    role=role,
                    face_encoding=avg_encoding,
                    enrolled_date=datetime.now().isoformat(),
                    photo_path=str(self.config.FACES_DIR / f"{name}_1.jpg")
                )
                
                # Save to database
                self.trusted_persons[name] = person
                self.save_database()
                
                print(f"\n🎉 SUCCESS! {name} enrolled as {role}")
                print(f"   Total samples: {len(captured_encodings)}")
                return True
            else:
                print("❌ Failed to capture enough samples")
                return False
                
        except Exception as e:
            print(f"\n❌ ERROR: {e}")
            import traceback
            traceback.print_exc()
            video_capture.release()
            cv2.destroyAllWindows()
            return False
    
    def save_database(self):
        """Save trusted persons database"""
        data = {name: person.to_dict() for name, person in self.trusted_persons.items()}
        with open(self.config.DB_FILE, 'wb') as f:
            pickle.dump(data, f)
        print(f"💾 Database saved: {len(self.trusted_persons)} persons")
    
    def load_database(self):
        """Load trusted persons database"""
        if self.config.DB_FILE.exists():
            with open(self.config.DB_FILE, 'rb') as f:
                data = pickle.load(f)
                self.trusted_persons = {
                    name: TrustedPerson.from_dict(person_data)
                    for name, person_data in data.items()
                }
            print(f"📂 Database loaded: {len(self.trusted_persons)} persons")
        else:
            print("📂 No existing database found. Starting fresh.")
    
    def list_enrolled(self):
        """Display all enrolled persons"""
        if not self.trusted_persons:
            print("📋 No persons enrolled yet.")
            return
        
        print(f"\n{'='*60}")
        print("👥 ENROLLED TRUSTED PERSONS")
        print(f"{'='*60}")
        for name, person in self.trusted_persons.items():
            print(f"  • {name} ({person.role})")
            print(f"    Enrolled: {person.enrolled_date[:10]}")
            print(f"    Recognitions: {person.recognition_count}")
        print(f"{'='*60}\n")
    
    def remove_person(self, name: str):
        """Remove a person from database"""
        if name in self.trusted_persons:
            del self.trusted_persons[name]
            self.save_database()
            print(f"✅ {name} removed from database")
        else:
            print(f"❌ {name} not found in database")

print("📝 Enrollment System ready!")

📝 Enrollment System ready!


In [14]:

# Cell 6: Face Recognition Engine
class FaceRecognitionEngine:
    """Real-time face recognition engine"""
    
    def __init__(self, enrollment_system: FaceEnrollmentSystem):
        self.enrollment_system = enrollment_system
        self.config = FaceRecognitionConfig()
        self.recognition_log = []
    
    def recognize_face(self, face_encoding: np.ndarray) -> Tuple[Optional[str], float]:
        """
        Recognize a face from its encoding
        
        Returns:
            (name, confidence) or (None, 0) if unknown
        """
        if not self.enrollment_system.trusted_persons:
            return None, 0.0
        
        # Get all known encodings and names
        known_encodings = [
            person.face_encoding 
            for person in self.enrollment_system.trusted_persons.values()
        ]
        known_names = list(self.enrollment_system.trusted_persons.keys())
        
        # Compare faces
        matches = face_recognition.compare_faces(
            known_encodings, 
            face_encoding, 
            tolerance=self.config.RECOGNITION_TOLERANCE
        )
        
        # Get face distances
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        
        # Find best match
        if True in matches:
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_names[best_match_index]
                confidence = 1.0 - face_distances[best_match_index]
                
                # Update recognition count
                self.enrollment_system.trusted_persons[name].recognition_count += 1
                
                return name, confidence
        
        return None, 0.0
    
    def process_frame(self, frame: np.ndarray) -> Tuple[np.ndarray, List[Dict]]:
        """
        Process a video frame for face recognition
        
        Returns:
            (annotated_frame, detections)
            detections: List of {name, role, confidence, location}
        """
        # Convert BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        
        detections = []
        annotated_frame = frame.copy()
        
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Recognize face
            name, confidence = self.recognize_face(face_encoding)
            
            if name:
                # Trusted person
                person = self.enrollment_system.trusted_persons[name]
                label = f"{name} ({person.role})"
                color = (0, 255, 0)  # Green
                
                detections.append({
                    'name': name,
                    'role': person.role,
                    'confidence': confidence,
                    'location': (top, right, bottom, left),
                    'trusted': True
                })
            else:
                # Unknown person
                label = "UNKNOWN INTRUDER"
                color = (0, 0, 255)  # Red
                
                detections.append({
                    'name': 'Unknown',
                    'role': 'intruder',
                    'confidence': 0.0,
                    'location': (top, right, bottom, left),
                    'trusted': False
                })
            
            # Draw rectangle
            cv2.rectangle(annotated_frame, (left, top), (right, bottom), color, 2)
            
            # Draw label background
            cv2.rectangle(annotated_frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
            
            # Draw label text
            cv2.putText(annotated_frame, label, (left + 6, bottom - 6),
                       cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)
            
            # Draw confidence
            if name:
                conf_text = f"{confidence:.2f}"
                cv2.putText(annotated_frame, conf_text, (left + 6, top - 6),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        return annotated_frame, detections
    
    def get_greeting(self, person_name: str) -> str:
        """Get Avengers-themed greeting for recognized person"""
        person = self.enrollment_system.trusted_persons.get(person_name)
        if not person:
            return "Access granted."
        
        import random
        greetings = self.config.PERSONALITY_GREETINGS.get(person.role, ["Welcome back."])
        return random.choice(greetings)
    
    def get_intruder_message(self, escalation_level: int = 1) -> str:
        """Get intruder warning message"""
        import random
        return random.choice(self.config.INTRUDER_MESSAGES)
    
    def log_detection(self, detection: Dict):
        """Log a detection event"""
        entry = {
            'timestamp': datetime.now().isoformat(),
            'name': detection['name'],
            'trusted': detection['trusted'],
            'confidence': detection.get('confidence', 0.0)
        }
        self.recognition_log.append(entry)

print("🔍 Recognition Engine ready!")

🔍 Recognition Engine ready!


In [77]:
# Cell 7: Live Recognition Demo (WITH EDGE TTS)
def run_live_recognition(duration: int = 30):
    """
    Run live face recognition from webcam
    
    Args:
        duration: How long to run (seconds)
    """
    print(f"\n{'🎥 '*20}")
    print("LIVE FACE RECOGNITION - DEMO MODE")
    print(f"{'🎥 '*20}\n")
    
    # Initialize systems
    enrollment = FaceEnrollmentSystem()
    engine = FaceRecognitionEngine(enrollment)
    
    # Initialize Edge TTS
    import edge_tts
    import asyncio
    import nest_asyncio
    import pygame
    import threading
    from queue import Queue
    import tempfile
    import os
    
    nest_asyncio.apply()
    pygame.mixer.init()
    
    speech_queue = Queue()
    audio_cache = {}
    
    def generate_audio(text):
        """Generate and cache audio file using Edge TTS"""
        if text in audio_cache:
            return audio_cache[text]
        
        try:
            async def _generate():
                communicate = edge_tts.Communicate(text, "en-US-GuyNeural")
                temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
                temp_file.close()
                await communicate.save(temp_file.name)
                return temp_file.name
            
            audio_file = asyncio.run(_generate())
            audio_cache[text] = audio_file
            return audio_file
        except Exception as e:
            print(f"❌ Audio generation error: {e}")
            return None
    
    def speech_worker():
        """Background worker for speech playback"""
        while True:
            text = speech_queue.get()
            if text is None:
                break
            try:
                print(f"🔊 SPEAKING: '{text}'")
                
                audio_file = generate_audio(text)
                
                if audio_file:
                    pygame.mixer.music.load(audio_file)
                    pygame.mixer.music.play()
                    
                    while pygame.mixer.music.get_busy():
                        pygame.time.Clock().tick(10)
                    
                    time.sleep(0.1)
                    pygame.mixer.music.unload()
                    print(f"✅ FINISHED SPEAKING")
                
            except Exception as e:
                print(f"❌ TTS Error: {e}")
            
            speech_queue.task_done()
    
    # Pre-generate common greetings
    print("🎵 Pre-generating audio files with JARVIS voice...")
    all_greetings = []
    for role_greetings in enrollment.config.PERSONALITY_GREETINGS.values():
        all_greetings.extend(role_greetings)
    all_greetings.extend(enrollment.config.INTRUDER_MESSAGES)
    
    for greeting in all_greetings:
        generate_audio(greeting)
    print(f"✅ {len(audio_cache)} audio files cached\n")
    
    # Start speech worker
    speech_thread = threading.Thread(target=speech_worker, daemon=True)
    speech_thread.start()
    
    def speak_async(text):
        speech_queue.put(text)
    
    if not enrollment.trusted_persons:
        print("⚠️  No trusted persons enrolled!")
        return
    
    enrollment.list_enrolled()
    
    # Open webcam
    video_capture = cv2.VideoCapture(0)
    
    if not video_capture.isOpened():
        print("❌ Could not access webcam!")
        return
    
    print(f"🎥 Starting recognition (running for {duration} seconds)...")
    print("Press 'q' to quit early\n")
    
    start_time = time.time()
    frame_count = 0
    detections_count = 0
    already_greeted = set()
    person_present = {}
    ABSENCE_THRESHOLD = 5
    
    try:
        while (time.time() - start_time) < duration:
            ret, frame = video_capture.read()
            if not ret:
                continue
            
            frame_count += 1
            current_time = time.time()
            seen_this_frame = set()
            
            if frame_count % 3 == 0:
                annotated_frame, detections = engine.process_frame(frame)
                
                for detection in detections:
                    person_id = detection.get('name', 'Unknown')
                    seen_this_frame.add(person_id)
                    person_present[person_id] = current_time
                    
                    should_greet = person_id not in already_greeted
                    
                    if should_greet:
                        if detection['trusted']:
                            greeting = engine.get_greeting(detection['name'])
                            print(f"✅ {detection['name']}: {greeting}")
                            speak_async(greeting)
                            already_greeted.add(person_id)
                        else:
                            warning = engine.get_intruder_message()
                            print(f"⚠️  INTRUDER: {warning}")
                            speak_async(warning)
                            already_greeted.add(person_id)
                        
                        detections_count += 1
                    
                    engine.log_detection(detection)
                
                for person_id in list(person_present.keys()):
                    if person_id not in seen_this_frame:
                        time_since_seen = current_time - person_present[person_id]
                        if time_since_seen > ABSENCE_THRESHOLD:
                            if person_id in already_greeted:
                                already_greeted.remove(person_id)
                                print(f"👋 {person_id} left the frame")
                            del person_present[person_id]
                
                cv2.imshow('Avengers Guard - Face Recognition (Press q to quit)', annotated_frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
    finally:
        video_capture.release()
        cv2.destroyAllWindows()
        print("\n⏳ Waiting for speech to complete...")
        speech_queue.join()
        speech_queue.put(None)
        speech_thread.join(timeout=5)
        
        # Cleanup cached audio files
        print("🧹 Cleaning up audio cache...")
        for audio_file in audio_cache.values():
            try:
                if os.path.exists(audio_file):
                    os.unlink(audio_file)
            except:
                pass
        
        pygame.mixer.quit()
        print("✅ Done")
    
    print(f"\n{'='*60}")
    print("📊 RECOGNITION SUMMARY")
    print(f"{'='*60}")
    print(f"Duration: {int(time.time() - start_time)} seconds")
    print(f"Frames processed: {frame_count}")
    print(f"Detections: {detections_count}")
    print(f"Recognition log entries: {len(engine.recognition_log)}")
    print(f"{'='*60}\n")
    
    return engine

In [78]:
# Cell 8: Test with Static Images (WITH EDGE TTS)
def test_recognition_from_images(test_image_paths: List[str]):
    """
    Test face recognition on static images with JARVIS voice
    """
    print(f"\n{'🖼️  '*20}")
    print("STATIC IMAGE RECOGNITION TEST")
    print(f"{'🖼️  '*20}\n")
    
    enrollment = FaceEnrollmentSystem()
    engine = FaceRecognitionEngine(enrollment)
    
    if not enrollment.trusted_persons:
        print("⚠️  No trusted persons enrolled!")
        return
    
    # Initialize Edge TTS
    import edge_tts
    import asyncio
    import nest_asyncio
    import pygame
    import threading
    from queue import Queue
    import tempfile
    import os
    
    nest_asyncio.apply()
    pygame.mixer.init()
    
    speech_queue = Queue()
    audio_cache = {}
    
    def generate_audio(text):
        """Generate and cache audio file using Edge TTS"""
        if text in audio_cache:
            return audio_cache[text]
        
        try:
            async def _generate():
                communicate = edge_tts.Communicate(text, "en-US-GuyNeural")
                temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
                temp_file.close()
                await communicate.save(temp_file.name)
                return temp_file.name
            
            audio_file = asyncio.run(_generate())
            audio_cache[text] = audio_file
            return audio_file
        except Exception as e:
            print(f"❌ Audio generation error: {e}")
            return None
    
    def speech_worker():
        """Background worker for speech playback"""
        while True:
            text = speech_queue.get()
            if text is None:
                break
            try:
                print(f"   🔊 SPEAKING: '{text}'")
                
                audio_file = generate_audio(text)
                
                if audio_file:
                    pygame.mixer.music.load(audio_file)
                    pygame.mixer.music.play()
                    
                    while pygame.mixer.music.get_busy():
                        pygame.time.Clock().tick(10)
                    
                    time.sleep(0.1)
                    pygame.mixer.music.unload()
                    print(f"   ✅ FINISHED SPEAKING")
                
            except Exception as e:
                print(f"   ❌ TTS Error: {e}")
            
            speech_queue.task_done()
    
    # Pre-generate common greetings
    print("🎵 Pre-generating audio files with JARVIS voice...")
    all_greetings = []
    for role_greetings in enrollment.config.PERSONALITY_GREETINGS.values():
        all_greetings.extend(role_greetings)
    all_greetings.extend(enrollment.config.INTRUDER_MESSAGES)
    
    for greeting in all_greetings:
        generate_audio(greeting)
    print(f"✅ {len(audio_cache)} audio files cached\n")
    
    # Start speech worker
    speech_thread = threading.Thread(target=speech_worker, daemon=True)
    speech_thread.start()
    
    def speak_async(text):
        speech_queue.put(text)
    
    results = []
    
    fig, axes = plt.subplots(1, len(test_image_paths), figsize=(5*len(test_image_paths), 5))
    if len(test_image_paths) == 1:
        axes = [axes]
    
    for idx, image_path in enumerate(test_image_paths):
        print(f"\n🖼️  Testing: {image_path}")
        
        try:
            frame = cv2.imread(image_path)
            if frame is None:
                print(f"❌ Could not load image: {image_path}")
                continue
            
            annotated_frame, detections = engine.process_frame(frame)
            
            for detection in detections:
                if detection['trusted']:
                    person = enrollment.trusted_persons[detection['name']]
                    greeting = engine.get_greeting(detection['name'])
                    print(f"✅ {detection['name']} ({person.role}) - Confidence: {detection['confidence']:.2f}")
                    print(f"   💬 {greeting}")
                    
                    speak_async(greeting)
                    
                    results.append(('trusted', detection['name'], detection['confidence']))
                else:
                    message = engine.get_intruder_message()
                    print(f"⚠️  UNKNOWN INTRUDER DETECTED")
                    print(f"   💬 {message}")
                    
                    speak_async(message)
                    
                    results.append(('intruder', 'Unknown', 0.0))
            
            rgb_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
            axes[idx].imshow(rgb_frame)
            axes[idx].set_title(f"Test {idx+1}")
            axes[idx].axis('off')
            
        except Exception as e:
            print(f"❌ Error processing {image_path}: {e}")
    
    # Wait for all speech to complete
    print("\n⏳ Waiting for speech to complete...")
    speech_queue.join()
    speech_queue.put(None)
    speech_thread.join(timeout=5)
    
    # Cleanup cached audio files
    print("🧹 Cleaning up audio cache...")
    for audio_file in audio_cache.values():
        try:
            if os.path.exists(audio_file):
                os.unlink(audio_file)
        except:
            pass
    
    pygame.mixer.quit()
    
    plt.tight_layout()
    plt.show()
    
    # Calculate accuracy
    trusted_count = sum(1 for r in results if r[0] == 'trusted')
    accuracy = (trusted_count / len(results) * 100) if results else 0
    
    print(f"\n{'='*60}")
    print("📊 TEST RESULTS")
    print(f"{'='*60}")
    print(f"Images tested: {len(test_image_paths)}")
    print(f"Trusted recognized: {trusted_count}")
    print(f"Intruders detected: {len(results) - trusted_count}")
    print(f"Recognition rate: {accuracy:.1f}%")
    print(f"{'='*60}\n")
    
    return results

In [79]:
# Cell 9: Quick Enrollment Helper
def quick_enroll_demo():
    """Quick helper to enroll yourself from webcam"""
    print("""
🚀 QUICK ENROLLMENT GUIDE

This will help you enroll faces into the system.
Choose your method:

1. From webcam (recommended):
   >>> enrollment = FaceEnrollmentSystem()
   >>> enrollment.enroll_from_webcam("YourName", "owner", num_samples=3)

2. From image file:
   >>> enrollment = FaceEnrollmentSystem()
   >>> enrollment.enroll_from_image("path/to/photo.jpg", "YourName", "owner")

3. Enroll roommate/friend:
   >>> enrollment.enroll_from_webcam("RoommateName", "roommate")
   >>> enrollment.enroll_from_webcam("FriendName", "friend")

4. Check enrolled persons:
   >>> enrollment.list_enrolled()
    """)
    
    return FaceEnrollmentSystem()

In [80]:

# Cell 10: Milestone 2 Validation
def validate_milestone_2():
    """
    Validates that Milestone 2 requirements are met:
    - Face detection working
    - Enrollment system functional
    - Recognition with 80%+ accuracy target
    - Proper handling of trusted vs unknown individuals
    """
    print("🔍 VALIDATING MILESTONE 2 REQUIREMENTS\n")
    
    enrollment = FaceEnrollmentSystem()
    
    checklist = {
        "✅ Face detection implemented": True,
        "✅ Face recognition with embeddings": True,
        "✅ Enrollment system (photo & webcam)": True,
        "✅ Trusted persons database": True,
        "✅ Real-time recognition engine": True,
        "✅ Welcome messages for trusted users": True,
        "✅ Intruder detection and warnings": True,
        "✅ Recognition logging": True
    }
    
    for item, status in checklist.items():
        print(f"{item}")
    
    print(f"\n{'='*60}")
    print("🎯 MILESTONE 2 STATUS: COMPLETE ✅")
    print(f"{'='*60}")
    
    if len(enrollment.trusted_persons) > 0:
        print(f"\n👥 Enrolled persons: {len(enrollment.trusted_persons)}")
        enrollment.list_enrolled()
    else:
        print("\n⚠️  No persons enrolled yet. Use quick_enroll_demo() to get started!")
    
    print("\n📝 Next Steps:")
    print("   → Enroll 1-2 trusted persons")
    print("   → Test with different lighting conditions")
    print("   → Record demo video showing recognition")
    print("   → Move to Milestone 3: Escalation Dialogue")
    print(f"{'='*60}\n")

validate_milestone_2()

🔍 VALIDATING MILESTONE 2 REQUIREMENTS

📂 Database loaded: 4 persons
✅ Face detection implemented
✅ Face recognition with embeddings
✅ Enrollment system (photo & webcam)
✅ Trusted persons database
✅ Real-time recognition engine
✅ Welcome messages for trusted users
✅ Intruder detection and warnings
✅ Recognition logging

🎯 MILESTONE 2 STATUS: COMPLETE ✅

👥 Enrolled persons: 4

👥 ENROLLED TRUSTED PERSONS
  • Mohith (owner)
    Enrolled: 2025-10-04
    Recognitions: 0
  • Damodar (roommate)
    Enrolled: 2025-09-30
    Recognitions: 0
  • Piyush (Friend)
    Enrolled: 2025-09-30
    Recognitions: 0
  • Arnav (Friend)
    Enrolled: 2025-09-30
    Recognitions: 0


📝 Next Steps:
   → Enroll 1-2 trusted persons
   → Test with different lighting conditions
   → Record demo video showing recognition
   → Move to Milestone 3: Escalation Dialogue



In [81]:

# Cell 11: Complete Testing Script
print("""
🎬 MILESTONE 2 - READY TO USE!

=== STEP-BY-STEP GUIDE ===

1️⃣ ENROLL YOURSELF:
>>> enrollment = FaceEnrollmentSystem()
>>> enrollment.enroll_from_webcam("YourName", "owner", num_samples=3)

2️⃣ ENROLL OTHERS (optional):
>>> enrollment.enroll_from_webcam("Roommate", "roommate")
>>> enrollment.enroll_from_image("friend_photo.jpg", "Friend", "friend")

3️⃣ TEST LIVE RECOGNITION:
>>> engine = run_live_recognition(duration=30)

4️⃣ TEST WITH IMAGES (for Colab):
>>> results = test_recognition_from_images(["test1.jpg", "test2.jpg"])

5️⃣ CHECK STATUS:
>>> enrollment.list_enrolled()
>>> validate_milestone_2()

=== TIPS ===
• Use good lighting for enrollment
• Capture 3+ samples for better accuracy  
• Test with different angles and lighting
• Adjust RECOGNITION_TOLERANCE in config if needed (default 0.6)

Ready to proceed? Run the cells above! 🚀
""")



🎬 MILESTONE 2 - READY TO USE!

=== STEP-BY-STEP GUIDE ===

1️⃣ ENROLL YOURSELF:
>>> enrollment = FaceEnrollmentSystem()
>>> enrollment.enroll_from_webcam("YourName", "owner", num_samples=3)

2️⃣ ENROLL OTHERS (optional):
>>> enrollment.enroll_from_webcam("Roommate", "roommate")
>>> enrollment.enroll_from_image("friend_photo.jpg", "Friend", "friend")

3️⃣ TEST LIVE RECOGNITION:
>>> engine = run_live_recognition(duration=30)

4️⃣ TEST WITH IMAGES (for Colab):
>>> results = test_recognition_from_images(["test1.jpg", "test2.jpg"])

5️⃣ CHECK STATUS:
>>> enrollment.list_enrolled()
>>> validate_milestone_2()

=== TIPS ===
• Use good lighting for enrollment
• Capture 3+ samples for better accuracy  
• Test with different angles and lighting
• Adjust RECOGNITION_TOLERANCE in config if needed (default 0.6)

Ready to proceed? Run the cells above! 🚀



In [82]:
enrollment = FaceEnrollmentSystem()
#enrollment.enroll_from_image( r"C:\Users\iammo\OneDrive\Desktop\Mohith_Personal\Pics\mypic1.jpg", "Mohith", "owner")
#enrollment.enroll_from_image( r"C:\Users\iammo\OneDrive\Desktop\Mohith_Personal\Pics\mypic4.jpg", "Mohith", "owner")


📂 Database loaded: 4 persons


In [14]:
enrollment.enroll_from_image( r"C:\Users\iammo\OneDrive\Desktop\Mohith_Personal\Pics\mypic3.jpg", "Mohith", "owner")



📸 ENROLLING: Mohith (owner)
✅ Image loaded: (1040, 694, 3)
💾 Database saved: 1 persons
✅ Mohith enrolled successfully!
   Role: owner
   Encoding shape: (128,)



True

In [15]:
enrollment.enroll_from_image( r"C:\Users\iammo\OneDrive\Desktop\Mohith_Personal\Pics\roomate.jpg", "Damodar", "roommate")



📸 ENROLLING: Damodar (roommate)
✅ Image loaded: (1280, 573, 3)
⚠️  Multiple faces detected (2). Using the largest face.
💾 Database saved: 2 persons
✅ Damodar enrolled successfully!
   Role: roommate
   Encoding shape: (128,)



True

In [16]:
enrollment.enroll_from_image( r"C:\Users\iammo\OneDrive\Desktop\Mohith_Personal\Pics\piyush.jpg", "Piyush", "Friend")



📸 ENROLLING: Piyush (Friend)
✅ Image loaded: (1280, 573, 3)
💾 Database saved: 3 persons
✅ Piyush enrolled successfully!
   Role: Friend
   Encoding shape: (128,)



True

In [12]:
enrollment.enroll_from_webcam("Mohith", "owner", num_samples=3)



📹 AUTO-ENROLLMENT: Mohith as owner
📋 Just look at the camera and STAY STILL
📋 System captures automatically every 2 seconds
📋 Press ESC to cancel

✅ Webcam opened
📸 Attempting capture 1...
   Detecting faces...
   Found 0 face(s)
   ⚠️ No face detected - move closer or improve lighting

📸 Attempting capture 1...
   Detecting faces...
   Found 1 face(s)
   Getting face encoding...
   ✅ Sample 1/3 captured!

📸 Attempting capture 2...
   Detecting faces...
   Found 1 face(s)
   Getting face encoding...
   ✅ Sample 2/3 captured!

📸 Attempting capture 3...
   Detecting faces...
   Found 1 face(s)
   Getting face encoding...
   ✅ Sample 3/3 captured!


📷 Closing webcam...
✅ Webcam closed

💾 Saving to database...
💾 Database saved: 4 persons

🎉 SUCCESS! Mohith enrolled as owner
   Total samples: 3


True

In [83]:
engine = run_live_recognition(duration=40)



🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 
LIVE FACE RECOGNITION - DEMO MODE
🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 

📂 Database loaded: 4 persons
🎵 Pre-generating audio files with JARVIS voice...
✅ 12 audio files cached


👥 ENROLLED TRUSTED PERSONS
  • Mohith (owner)
    Enrolled: 2025-10-04
    Recognitions: 0
  • Damodar (roommate)
    Enrolled: 2025-09-30
    Recognitions: 0
  • Piyush (Friend)
    Enrolled: 2025-09-30
    Recognitions: 0
  • Arnav (Friend)
    Enrolled: 2025-09-30
    Recognitions: 0

🎥 Starting recognition (running for 40 seconds)...
Press 'q' to quit early

✅ Mohith: Access granted. Room status: secured.
🔊 SPEAKING: 'Access granted. Room status: secured.'
✅ FINISHED SPEAKING
👋 Mohith left the frame
✅ Mohith: Access granted. Room status: secured.
🔊 SPEAKING: 'Access granted. Room status: secured.'
✅ FINISHED SPEAKING
⚠️  INTRUDER: Access denied. You are not in my database of trusted individuals.
🔊 SPEAKING: 'Access denied. You are not in my database of trusted in

In [13]:
enrollment.enroll_from_image( r"C:\Users\iammo\OneDrive\Desktop\Mohith_Personal\Pics\arnav.jpg", "Arnav", "Friend")



📸 ENROLLING: Arnav (Friend)
✅ Image loaded: (1137, 1111, 3)
💾 Database saved: 4 persons
✅ Arnav enrolled successfully!
   Role: Friend
   Encoding shape: (128,)



True

In [19]:
# Test recognition with greetings
engine = run_live_recognition(duration=30)


🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 
LIVE FACE RECOGNITION - DEMO MODE
🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 🎥 

📂 Database loaded: 4 persons

👥 ENROLLED TRUSTED PERSONS
  • Mohith (owner)
    Enrolled: 2025-09-30
    Recognitions: 0
  • Damodar (roommate)
    Enrolled: 2025-09-30
    Recognitions: 0
  • Piyush (Friend)
    Enrolled: 2025-09-30
    Recognitions: 0
  • Arnav (Friend)
    Enrolled: 2025-09-30
    Recognitions: 0

🎥 Starting recognition (running for 30 seconds)...
Press 'q' to quit early

✅ Mohith: Access granted. Room status: secured.
✅ Mohith: Access granted. Room status: secured.
✅ Mohith: Good to see you, sir. All systems nominal.
✅ Mohith: Access granted. Room status: secured.
✅ Mohith: Welcome back, boss. JARVIS has kept everything secure.
✅ Mohith: Welcome back, boss. JARVIS has kept everything secure.
✅ Mohith: Good to see you, sir. All systems nominal.
✅ Mohith: Access granted. Room status: secured.
✅ Mohith: Access granted. Room status: secured.
✅ Mohith: W

In [52]:
import pyttsx3
import time

engine = pyttsx3.init()
engine.setProperty('rate', 175)

print("Speaking message 1...")
engine.say("Access granted. Room status: secured.")
engine.runAndWait()
print("Finished message 1")

time.sleep(2)  # Wait 2 seconds

print("Speaking message 2...")
engine.say("Welcome back, boss. JARVIS has kept everything secure.")
engine.runAndWait()
print("Finished message 2")

print("Done!")

Speaking message 1...
Finished message 1
Speaking message 2...
Finished message 2
Done!
